In [38]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

In [1]:
import pandas as pd
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

# Ejercicio


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio debeis entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso podréis centraros en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.


**Nota 1**: Este ejercicio requiere el uso de las GPUs de Google Colab. Este Colab debería estar preconfigurado para ejecutarse en GPU, pero si tuviera problemas en la ejecución que me contacte a través del Moodle para buscar soluciones alternativas.

## 0. Imports


In [19]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW
)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import accelerate
import numpy as np
import evaluate
from sklearn.metrics import classification_report, confusion_matrix

## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [3]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

In [4]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preprocesado y Normalización

#### En el ejercicio anterior habíamos identificado algunos duplicados que deberemos eliminar en este ejercicio también. Deberemos tener en cuenta que había dos tipos de eliminación de duplicados, los que se repetían en ambas columnas, y los que tenían etiquetas contradictorias.

In [5]:
dataset = dataset.drop_duplicates(subset=["tweet_text", 'cyberbullying'], keep = 'first')

print("Después de la limpieza, quedan {} documentos".format(len(dataset)))

Después de la limpieza, quedan 47526 documentos


In [6]:
dataset = dataset.drop_duplicates(subset = ['tweet_text'], keep = False)
print('Después de eliminar todas las filas con ambas etiquetas, quedan {} documentos'.format(len(dataset)))

Después de eliminar todas las filas con ambas etiquetas, quedan 44508 documentos


In [7]:
#Separamos las etiquetas de los tweets
texts  = dataset['tweet_text'].values
labels = dataset['cyberbullying'].values

In [8]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.25, random_state=0,
                                                    stratify = labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=0,stratify = train_labels)

In [9]:
model_name = 'bert-base-uncased'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#### Comprobamos que el tokenizador funciona correctamente con el primer tweet de nuestro dataset.

In [11]:
texto = dataset['tweet_text'].sample(1).iloc[0]
texto_tokens = tokenizer(texto).tokens()
texto_tokens

['[CLS]',
 'missing',
 'school',
 'for',
 'the',
 'grey',
 'cup',
 'festival',
 '##l',
 'at',
 'nathan',
 'phillips',
 'square',
 '#',
 '86',
 '##in',
 '##weight',
 '##train',
 '##ing',
 '#',
 'mar',
 '##ved',
 '#',
 'food',
 '#',
 'w',
 '##hee',
 '##ee',
 '##ee',
 '##ew',
 '[SEP]']

In [12]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """

        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [13]:
max_length = 125

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)

In [14]:
max_seq_length = 96
train_batch_size =  8
eval_batch_size = 8
test_batch_size = 8

## 5. Entrenamiento y evaluación de modelos


In [15]:


id2label = {0: "Non_cyberbullying", 1: "Cyberbullying"}
label2id = {"Non_cyberbullying": 0, "Cyberbullying": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

In [18]:
accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [21]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.241000,0.262438,{'accuracy': 0.9213718735959263},{'f1': 0.9545965579866816}
2,0.174900,0.295836,{'accuracy': 0.9224202486146473},{'f1': 0.9557718579234974}
3,0.111600,0.333867,{'accuracy': 0.9258649093904449},{'f1': 0.9571242962321352}
4,0.061800,0.433915,{'accuracy': 0.9248165343717238},{'f1': 0.9566044260027663}


Trainer is attempting to log a value of "{'accuracy': 0.9213718735959263}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9545965579866816}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9224202486146473}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9557718579234974}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9258649093904449}" of type <clas

TrainOutput(global_step=13352, training_loss=0.15975687045262488, metrics={'train_runtime': 2955.0953, 'train_samples_per_second': 36.146, 'train_steps_per_second': 4.518, 'total_flos': 6861442990560000.0, 'train_loss': 0.15975687045262488, 'epoch': 4.0})

In [22]:
predictions = trainer.predict(test_dataset)

In [23]:
predictions[0][0]

array([-3.2565544,  3.8045218], dtype=float32)

In [24]:
y_pred = predictions.predictions.argmax(axis=1)

In [25]:
y_true = [x["labels"].item() for x in test_dataset]

In [26]:
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))

[[1049  558]
 [ 354 9166]]
              precision    recall  f1-score   support

           0       0.75      0.65      0.70      1607
           1       0.94      0.96      0.95      9520

    accuracy                           0.92     11127
   macro avg       0.85      0.81      0.82     11127
weighted avg       0.91      0.92      0.92     11127



#### Como podemos observar los resultados obtenidos entrenando el modelo con la biblioteca de Transformers es claramente mejor, con una precisión de 0`75 en los casos de tweets sin cyberacoso y una precisión de 0`94 en los tweets con cyberacoso. El recall también ha mejorado con respecto al modelo del ejercicio anterior.

#### Debemos tener en cuenta lo que mejora el modelo con esta librería, ya que los datos no están balanceados y en el caso del ejercicio anterior si.